In [1]:
import sys
sys.path.append('../src_variable')

import network as nw
import torch
import generate_kvadraturen as gk
import bus
import road as rd
import traffic_lights as tl
import junction as jn
import numpy as np

In [2]:
# torch.autograd.set_detect_anomaly(True)

In [3]:
T = 400
network = gk.generate_kvadraturen_small(T)
# network = gk.generate_kvadraturen_wo_tl(T)

ids_bw = ["lundsbro_bw", "elvegata_bw", "tollbod_6bw", "tollbod_5bw", "tollbod_4bw", 
        "tollbod_3bw", "tollbod_2bw", "tollbod_1bw", "v_strand_7bw", "v_strand_6bw",
            "v_strand_5bw", "v_strand_4bw", "v_strand_3bw", "v_strand_2bw", "v_strand_1bw"]            
            

stops_bw = [("tollbod_6bw", 50), ("tollbod_3bw", 90), ("tollbod_1bw", 30), ("v_strand_3bw", 25)]
# stops_bw = [("tollbod_6bw", 50), ("tollbod_3bw", 90)]

times_bw = [40, 130, 190, 250]
# times_bw = [10, 70]

bus_bw = bus.Bus(ids_bw, stops_bw, times_bw, network)

ids_fw = ["v_strand_1fw", "v_strand_2fw", "v_strand_3fw", "h_w_2", "h_w_3",
    "h_w_4", "festning_4fw", "festning_5fw", "festning_6fw", "festning_7fw",
    "tollbod_4fw", "tollbod_5fw", "tollbod_6fw", "elvegata_fw", "lundsbro_fw"]

stops_fw = [("v_strand_1fw", 40), ("h_w_3", 30), ("festning_5fw", 40), ("tollbod_4fw", 25), 
            ("tollbod_6fw", 60)]
# stops_fw = [("h_w_3", 30), ("festning_5fw", 40), ("tollbod_4fw", 25), 
#             ("tollbod_6fw", 60)]
# stops_fw = [("h_w_3", 30), ("festning_5fw", 40), ("tollbod_4fw", 10)]
times_fw = [1, 30, 110, 130, 230]
# times_fw = [10, 40, 80, 120]
# times_fw = [10, 40, 70]

bus_fw = bus.Bus(ids_fw, stops_fw, times_fw, network)

In [4]:
# Create actual network
roads = network.roads
junctions = network.junctions
T = network.T
# Don't store the densities 
# bus_network = nw.RoadNetwork(roads, junctions, T, [bus_fw, bus_bw], store_densities = False)
bus_network = nw.RoadNetwork(roads, junctions, T, [bus_fw, bus_bw], store_densities = False)

In [5]:
# with torch.autograd.set_detect_anomaly(True):
#     densities, queues, bus_lengths, bus_delays = bus_network.solve_cons_law()
densities, queues, bus_lengths, bus_delays = bus_network.solve_cons_law()

In [6]:
print(bus_delays)

{0: [tensor(2.5116, grad_fn=<AddBackward0>), tensor(0., grad_fn=<AddBackward0>), tensor(0., grad_fn=<AddBackward0>), tensor(58.6466, grad_fn=<AddBackward0>), tensor(17.1148, grad_fn=<AddBackward0>)], 1: [tensor(0., grad_fn=<AddBackward0>), tensor(0., grad_fn=<AddBackward0>), tensor(0., grad_fn=<AddBackward0>), tensor(117.3570, grad_fn=<AddBackward0>)]}


In [7]:
# Cloning just to be safe
# objective = bus_delays[0][0].clone() #+ bus_delays[1][0].clone()
# for i in range(len(bus_delays)):
#     for j in range(len(bus_delays[i])):
#         if i == 0 and j == 0:
#             continue
#         objective = objective + bus_delays[i][j].clone()
# objective = bus_delays[0][0] + bus_delays[0][1] + bus_delays[0][2]
objective = torch.tensor(0.0)

# Error with stop number 3 and 4 for fw bus and stop 4 for bw bus

# objective = bus_delays[0][0]  
for i in range(len(bus_delays)):
    for j in range(len(bus_delays[i])):
        # if j == 2:
        #     continue
        objective = objective + bus_delays[i][j]

In [8]:
print(objective)

tensor(195.6299, grad_fn=<AddBackward0>)


In [9]:
# objective.backward(retain_graph=True)
objective.backward(retain_graph=True)

In [10]:
params = [v for road in bus_network.roads for v in road.Vmax] +\
        [t for junction in bus_network.junctions for traffic_light in junction.trafficlights for t in traffic_light.cycle] +\
        [t for junction in bus_network.junctions for traffic_light in junction.coupled_trafficlights for t in traffic_light.cycle]

In [11]:
print("Road Vmax gradients:")
for road in bus_network.roads:
    for v in road.Vmax:
        print(v.grad)

Road Vmax gradients:
tensor(-0.2147)
tensor(5.1381e-09)
tensor(1.5925e-07)
tensor(2.1815e-05)
tensor(2.4246e-05)
tensor(-7.8203e-05)
tensor(7.5396e-05)
tensor(7.5433e-05)
tensor(-7.1640e-11)
tensor(-2.9962e-11)
tensor(-0.1284)
tensor(-0.0035)
tensor(-4.7124e-05)
tensor(-0.0002)
tensor(0.0002)
tensor(-8.8593e-06)
tensor(0.0002)
tensor(2.1430e-10)
tensor(-2.1953e-09)
tensor(0.)
tensor(-1.8762)
tensor(-1.4539)
tensor(-0.8621)
tensor(-0.0010)
tensor(-0.0007)
tensor(-0.0007)
tensor(-0.0008)
tensor(-0.0006)
tensor(-0.0006)
tensor(-4.5511e-07)
tensor(-0.0003)
tensor(0.0001)
tensor(-0.0016)
tensor(0.0001)
tensor(-3.3369e-06)
tensor(-1.2600e-05)
tensor(-9.5367e-05)
tensor(3.3264e-05)
tensor(2.0803e-06)
tensor(-0.0314)
tensor(-0.1452)
tensor(-0.0166)
tensor(0.0005)
tensor(-3.4343e-07)
tensor(-6.4915e-07)
tensor(-0.0007)
tensor(-2.7000e-06)
tensor(0.0174)
tensor(-0.0001)
tensor(0.0040)
tensor(-0.0435)
tensor(-0.0001)
tensor(-8.4380e-05)
tensor(-0.0016)
tensor(-0.0007)
tensor(-0.0306)
tensor(-0.01

In [12]:
for junction in bus_network.junctions:
    for traffic_light in junction.trafficlights:
        for t in traffic_light.cycle:
            print(t.grad)

    for traffic_light in junction.coupled_trafficlights:
        for t in traffic_light.cycle:
            print(t.grad)

tensor(2.7797e-06)
tensor(3.3811e-06)
tensor(-0.0002)
tensor(3.9738e-05)
tensor(0.0001)
tensor(9.1615e-05)
tensor(0.0001)
tensor(-3.3951e-05)
tensor(-0.0004)
tensor(8.7675e-10)
tensor(0.0001)
tensor(-8.0652e-09)
tensor(0.0005)
tensor(7.1534e-09)
tensor(-0.0001)
tensor(3.6390e-26)
tensor(1.6812e-05)
tensor(-2.9619e-06)
tensor(-0.0005)
tensor(9.5887e-05)
tensor(0.0004)
tensor(4.2679e-24)
tensor(7.4302e-05)
tensor(7.4316e-05)
tensor(0.0005)
tensor(0.0006)


In [13]:
for j in bus_network.junctions:
    print(j.get_activation(195.0, "tollbod_4fw", "tollbod_5fw"))
    # print(j.get_activation(195.0, "festning_7fw", "tollbod_4fw"))

    

(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(True, 1.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
(False, 0.0)
